In [1]:
!pip install pymongo

     |████████████████████████████████| 512 kB 768 kB/s eta 0:00:01


In [2]:
import os
os.environ["PYSPARK_PYTHON"]="python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.7"

In [3]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
            .master("spark://spark-master:7077") \
            .appName("Mongo") \
            .getOrCreate()

In [4]:
import json
from pymongo import MongoClient

In [5]:
files = ['{:0>2}'.format(str(i)) for i in list(range(10, 11))]

In [6]:
client = MongoClient('mongodb://172.16.0.4:27017/')

In [7]:
db = client.bigdata

In [8]:
collection = db.apps

In [9]:
for file in files:
    df = spark.read.format("json").load("hdfs://namenode/user/root/input/data{name}.json".format(name=file), multiLine = "true")
    df = df.na.drop().dropDuplicates()
    df = df.filter(size(df['comments']) >= 30)
    
    df_json = df.toJSON()
    for row in df_json.collect():
        line = json.loads(row) 
        collection.insert_one(line)